# Analisando o bot

Este jupyter-notebook vai auxiliar na análise de detecção de intenções da Taís.

## Instalação

### Configurando jupyter e instalando as dependências do bot

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

### Listando versões

In [2]:
import rasa_nlu

print("rasa_nlu: {}".format(rasa_nlu.__version__))

rasa_nlu: 0.14.6


### Treinando o modelo do Rasa NLU

Abaixo será feito o treinamento do Rasa NLU com as intents do coach. Em seguida será feita a avaliação das intents treinadas.

In [6]:
import os

from rasa_nlu.training_data import load_data
from rasa_nlu.model import Trainer
from rasa_nlu import config


intents_directory = '../../bot/data/intents/'

# loading the nlu training samples
training_data = load_data(intents_directory)

# trainer to educate our pipeline
trainer = Trainer(config.load("../../bot/nlu_config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/processo.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 240 (19 distinct intents)
	- Found intents: 'processo_prazo_desarquivar', 'processo_prazo_readequacao', 'processo_reativacao_de_proposta', 'processo_prazo_apresentar_proposta', 'processo_prazo_analise_proposta', 'processo_preenchimento', 'processo_analise_de_resultados', 'processo_prazo_periodo_captacao', 'processo_prazo_desistir_recurso', 'processo_aprovacao', 'processo_prazo_envio_cnae', 'processo_definicao_etapas', 'processo_admissibilidade', 'processo_prazo', 'processo_como_funciona', 'processo_prazo_analise_tecnica', 'processo_prazo_prestacao_contas', 'processo_execucao', 'processo_prazo_diligencias'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/lei_rouanet.md is md
INFO:rasa_nlu.training_data.train

In [7]:
from rasa_nlu.evaluate import run_evaluation
evaluation = run_evaluation('../../bot/data/intents/', model_directory)

INFO:tensorflow:Restoring parameters from /work/notebooks/intents/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt
INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/processo.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 240 (19 distinct intents)
	- Found intents: 'processo_prazo_desarquivar', 'processo_prazo_readequacao', 'processo_reativacao_de_proposta', 'processo_prazo_apresentar_proposta', 'processo_prazo_analise_proposta', 'processo_preenchimento', 'processo_analise_de_resultados', 'processo_prazo_periodo_captacao', 'processo_prazo_desistir_recurso', 'processo_aprovacao', 'processo_prazo_envio_cnae', 'processo_definicao_etapas', 'processo_admissibilidade', 'processo_prazo', 'processo_como_funciona', 'processo_prazo_analise_tecnica', 'processo_prazo_prestacao_contas', 'processo_execucao', 'processo_prazo_diligencias'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

INFO:rasa_nlu.evaluate:Model prediction errors saved to errors.json.
INFO:rasa_nlu.evaluate:Entity evaluation results:
INFO:rasa_nlu.evaluate:Evaluation for entity extractor: ner_crf 
INFO:rasa_nlu.evaluate:F1-Score:  0.9991955868865754
INFO:rasa_nlu.evaluate:Precision: 0.9992211877261385
INFO:rasa_nlu.evaluate:Accuracy:  0.9992205767731879
INFO:rasa_nlu.evaluate:Classification report: 
              precision    recall  f1-score   support

 lei_rouanet       1.00      0.88      0.94        33
   no_entity       1.00      1.00      1.00      5099

   micro avg       1.00      1.00      1.00      5132
   macro avg       1.00      0.94      0.97      5132
weighted avg       1.00      1.00      1.00      5132



## Problemas encontrados:

Veja o relatorio gerado pela célula abaixo, nele é possível identificar facilmente as intents com problemas de na previsão, isso significa que os exemplos dos textos de usuário dessas intents não estão bem definidos o que faz o bot entrar em confusão.

Para uma intent ser confundida com outra, significa que estão muito siilares entre si, uma melhor forma de concertar este erro é definido exemplos mais distintos para cada uma delas.

In [8]:
import json

try:
    f = open('errors.json', 'r')
    errors_list = json.load(f)

    for error in errors_list:
        print('Intent com erros de previsão: {}'.format(error['intent']))
        print('Intent com qual está sendo confundida: {}'.format(error['intent_prediction']['name']))
        print('Texto em que ocorre o erro: {}'.format(error['text']))
        print('Confiança da previsão: {}'.format(error['intent_prediction']['confidence']))
        print()
except(FileNotFoundError):
    print('Não foi encontrado nenhum erro de confusão entre as intents')
    


Intent com erros de previsão: processo_como_funciona
Intent com qual está sendo confundida: processo_prazo_apresentar_proposta
Texto em que ocorre o erro: enviar proposta
Confiança da previsão: 0.7544613480567932

Intent com erros de previsão: processo_prazo_apresentar_proposta
Intent com qual está sendo confundida: processo_como_funciona
Texto em que ocorre o erro: enviar projeto
Confiança da previsão: 0.8119997382164001



### Avaliação do modelo de NLU treinado

Caso queira testar mensagens específicas mande a mensagem na célula seguinte:

In [9]:
a = input("Escreva uma frase para testar a previsão do bot: ")
pprint(interpreter.parse(a))

Escreva uma frase para testar a previsão do bot: o que é a lei Rouanet?
{
  "intent": {
    "name": "lei_rouanet_o_que_eh",
    "confidence": 0.9461674690246582
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "lei_rouanet_o_que_eh",
      "confidence": 0.9461674690246582
    },
    {
      "name": "salic_preenchimento_campo_custo_auditoria",
      "confidence": 0.305629163980484
    },
    {
      "name": "lei_rouanet_analise_tecnica",
      "confidence": 0.2544594407081604
    },
    {
      "name": "processo_prazo_prestacao_contas",
      "confidence": 0.2459997534751892
    },
    {
      "name": "lei_rouanet_etapas_aprovacao_projeto",
      "confidence": 0.2458021342754364
    },
    {
      "name": "processo_prazo_analise_tecnica",
      "confidence": 0.2428033947944641
    },
    {
      "name": "salic_preenchimento_vinculo_cpf_proposta",
      "confidence": 0.23044267296791077
    },
    {
      "name": "salic_cadastro_proponente",
      "confidence": 0.22884619